In [ ]:
!pip install wikipedia-api tqdm
import requests, json, time
import wikipediaapi
from tqdm import tqdm

# Khởi tạo API tiếng Việt với user_agent hợp lệ
wiki = wikipediaapi.Wikipedia(
    language='vi',
    user_agent='NLP_FinalProject/1.0 (Contact: caoky.sonha@gmail.com)'
)

NUM_PAGES = 10000
topics = []

# Dùng tqdm để hiển thị tiến trình crawl
for i in tqdm(range(NUM_PAGES // 10), desc="📘 Đang lấy bài Wikipedia"):
    try:
        res = requests.get(
            "https://vi.wikipedia.org/w/api.php",
            params={
                "action": "query",
                "list": "random",
                "rnnamespace": 0,  # namespace=0 => bài chính
                "rnlimit": 10,
                "format": "json"
            },
            headers={"User-Agent": "NLP_FinalProject/1.0 (Contact: caoky.sonha@gmail.com)"}
        )
        data = res.json()

        # Ghi log mỗi 100 vòng
        if i % 100 == 0 and i > 0:
            print(f"Đã xử lý {i*10} bài... (tổng hiện tại: {len(topics)})")

        for page in data.get("query", {}).get("random", []):
            topics.append(page["title"])

        time.sleep(0.3)  # tránh spam API
    except Exception as e:
        print(f"⚠️ Lỗi ở vòng {i}: {e}")
        time.sleep(1)

print(f"\n✅ Hoàn tất! Lấy ngẫu nhiên được {len(topics)} bài viết.")

# Lưu danh sách tiêu đề để dùng lại
with open("wiki_topics.json", "w", encoding="utf-8") as f:
    json.dump(topics, f, ensure_ascii=False, indent=2)
print("💾 Đã lưu danh sách chủ đề vào file 'wiki_topics.json'")

In [ ]:
# Tải danh sách topics đã lưu
with open("/kaggle/working/wiki_topics.json", encoding="utf-8") as f:
    topics = json.load(f)

# Khởi tạo Wikipedia API (vẫn giữ user_agent)
wiki = wikipediaapi.Wikipedia(
    language='vi',
    user_agent='NLP_FinalProject/1.0 (Contact: caoky.sonha@gmail.com)'
)

contexts = []

# Crawl từng bài, có thanh tiến trình
for topic in tqdm(topics, desc="📄 Đang lấy nội dung bài Wikipedia"):
    try:
        page = wiki.page(topic)
        if not page.exists():
            continue

        text = page.text.strip()
        if not text:
            continue

        # Cắt bài thành đoạn
        paragraphs = text.split('\n')
        for p in paragraphs:
            words = p.split()
            if 40 <= len(words) <= 150: 
                contexts.append({
                    "topic": topic,
                    "context": p.strip()
                })

        # log định kỳ
        if len(contexts) % 1000 == 0:
            print(f"Đã thu được {len(contexts)} đoạn...")

        time.sleep(0.2)

    except Exception as e:
        print(f"⚠️ Lỗi ở bài '{topic}': {e}")
        time.sleep(0.5)

print(f"\n✅ Hoàn tất! Thu được {len(contexts)} đoạn văn usable từ {len(topics)} bài.")

# Lưu file kết quả
with open("raw_contexts_large.json", "w", encoding="utf-8") as f:
    json.dump(contexts, f, ensure_ascii=False, indent=2)

print("Đã lưu dữ liệu vào 'raw_contexts_large.json'")